<a href="https://colab.research.google.com/github/vicentcamison/idal_ia3/blob/main/3%20Aprendizaje%20profundo%20(II)/Sesion%205/2_RNN_chars_PT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![IDAL](https://i.imgur.com/tIKXIG1.jpg)  

#**Máster en Inteligencia Artificial Avanzada y Aplicada:  IA^3**
---

#<strong><center>Predicción de caracteres con RNNs y PyTorch</center></strong>

En este cuaderno implementaremos un sencillo modelo de predicción de carácteres con RNN para familiarizarnos con las funciones de PyTorch para estas redes e iniciarnos en las RNN.

En esta implementación, construiremos un modelo que pueda completar su frase basándose en unos pocos caracteres o en una palabra utilizada como entrada.
![Ejemplo](https://i.imgur.com/yA5xVhn.jpg)

Para mantener esto corto y simple, no usaremos ningún conjunto de datos grande o externo. En su lugar, nos limitaremos a definir unas pocas frases para ver cómo el modelo aprende de estas frases. El proceso que seguirá esta implementación es el siguiente:

![Resumen](https://i.imgur.com/KfvOIt5.jpg)

Empezaremos importando el paquete principal de PyTorch junto con la clase *Variable* utilizada para almacenar nuestros tensores de datos y el paquete *nn* que utilizaremos al construir el modelo. Además, sólo usaremos numpy para preprocesar nuestros datos ya que Torch funciona muy bien con numpy.

In [ ]:
import torch
from torch import nn

import numpy as np

En primer lugar, definiremos las frases que queremos que nuestro modelo produzca cuando se alimente con la primera palabra o los primeros caracteres.

Luego crearemos un diccionario con todos los caracteres que tenemos en las frases y los mapearemos a un entero. Esto nos permitirá convertir nuestros caracteres de entrada a sus respectivos enteros (*char2int*) y viceversa (*int2char*).

In [ ]:
text = ['hola que tal','bueno estoy bien','que tengas un buen día', 'espero que lo pases bien']
#text = ['hey how are you','good i am fine','have a nice day']

# Join all the sentences together and extract the unique characters from the combined sentences
chars = set(''.join(text))

# Creating a dictionary that maps integers to the characters
int2char = dict(enumerate(chars))

# Creating another dictionary that maps characters to integers
char2int = {char: ind for ind, char in int2char.items()}

In [ ]:
print(char2int)

{'g': 0, 'b': 1, 'a': 2, 't': 3, 'l': 4, 'p': 5, 'r': 6, 'í': 7, ' ': 8, 's': 9, 'd': 10, 'n': 11, 'y': 12, 'i': 13, 'q': 14, 'h': 15, 'e': 16, 'u': 17, 'o': 18}


A continuación, rellenaremos nuestras frases de entrada para asegurarnos de que todas las frases tienen la longitud de la muestra. Aunque las RNN suelen ser capaces de aceptar entradas de tamaño variable, normalmente querremos alimentar los datos de entrenamiento en lotes para acelerar el proceso de entrenamiento. Para poder utilizar lotes para entrenar con nuestros datos, tendremos que asegurarnos de que cada secuencia dentro de los datos de entrada tenga el mismo tamaño.

Por lo tanto, en la mayoría de los casos, el relleno puede hacerse rellenando las secuencias que son demasiado cortas con valores **0** y recortando las secuencias que son demasiado largas. En nuestro caso, encontraremos la longitud de la secuencia más larga y rellenaremos el resto de las frases con espacios en blanco para que coincidan con esa longitud.

In [ ]:
maxlen = len(max(text, key=len))+1
print("The longest string has {} characters".format(maxlen))

The longest string has 25 characters


In [ ]:
# Padding

# A simple loop that loops through the list of sentences and adds a ' ' whitespace until the length of the sentence matches
# the length of the longest sentence
for i in range(len(text)):
    while len(text[i])<maxlen:
        text[i] += ' '

Como vamos a predecir el siguiente carácter de la secuencia en cada paso de tiempo, tendremos que dividir cada frase en

- Datos de entrada
    - El último carácter de entrada debe excluirse, ya que no es necesario introducirlo en el modelo
- Etiqueta objetivo/de verdad
    - Un paso de tiempo por delante de los datos de entrada, ya que será la "respuesta correcta" para el modelo en cada paso de tiempo correspondiente a los datos de entrada

In [ ]:
# Creating lists that will hold our input and target sequences
input_seq = []
target_seq = []

for i in range(len(text)):
    # Remove last character for input sequence
    input_seq.append(text[i][:-1])
    
    # Remove firsts character for target sequence
    target_seq.append(text[i][1:])
    print("Input Sequence: {}\nTarget Sequence: {}".format(input_seq[i], target_seq[i]))

Input Sequence: hola que tal            
Target Sequence: ola que tal             
Input Sequence: bueno estoy bien        
Target Sequence: ueno estoy bien         
Input Sequence: que tengas un buen día  
Target Sequence: ue tengas un buen día   
Input Sequence: espero que lo pases bien
Target Sequence: spero que lo pases bien 


Ahora podemos convertir nuestras secuencias de entrada y de destino en secuencias de enteros en lugar de caracteres, mapeándolas con los diccionarios que creamos anteriormente. Esto nos permitirá codificar de una sola vez nuestra secuencia de entrada.

In [ ]:
for i in range(len(text)):
    input_seq[i] = [char2int[character] for character in input_seq[i]]
    target_seq[i] = [char2int[character] for character in target_seq[i]]

Antes de codificar nuestra secuencia de entrada en vectores de un solo golpe, definiremos 3 variables clave:

- *dict_size*: El número de caracteres únicos que tenemos en nuestro texto.
    - Esto determinará el tamaño del vector de un solo paso, ya que cada carácter tendrá un índice asignado en ese vector.
- *seq_len*: La longitud de las secuencias que estamos introduciendo en el modelo
    - Como hemos estandarizado la longitud de todas nuestras frases para que sean iguales a las frases más largas, este valor será la longitud máxima - 1 ya que también hemos eliminado el último carácter introducido
- *tamaño_del_lote*: El número de frases que hemos definido y que vamos a introducir en el modelo como un lote

In [ ]:
dict_size = len(char2int)
seq_len = maxlen - 1
batch_size = len(text)

def one_hot_encode(sequence, dict_size, seq_len, batch_size):
    # Creating a multi-dimensional array of zeros with the desired output shape
    features = np.zeros((batch_size, seq_len, dict_size), dtype=np.float32)
    
    # Replacing the 0 at the relevant character index with a 1 to represent that character
    for i in range(batch_size):
        for u in range(seq_len):
            features[i, u, sequence[i][u]] = 1
    return features

También definimos una función de ayuda que crea matrices de ceros para cada carácter y sustituye el índice del carácter correspondiente por un **1**.

In [ ]:
input_seq = one_hot_encode(input_seq, dict_size, seq_len, batch_size)
print("Input shape: {} --> (Batch Size, Sequence Length, One-Hot Encoding Size)".format(input_seq.shape))

Input shape: (4, 24, 19) --> (Batch Size, Sequence Length, One-Hot Encoding Size)


Ya que hemos terminado con todo el preprocesamiento de datos, ahora podemos mover los datos de los arrays de numpy a la propia estructura de datos de PyTorch - **Torch Tensors**.

In [ ]:
input_seq = torch.from_numpy(input_seq)
target_seq = torch.Tensor(target_seq)

Vamos ahora a la parte interesante. Después de preparar los datos, pasamos a definir el modelo. Definiremos el modelo usando la librería Torch, y aquí es donde puedes añadir o quitar capas, ya sean capas totalmente conectadas, capas convolucionales, capas RNN vainilla, capas LSTM, ¡y muchas más! En este ejercicio, utilizaremos el modelo básico nn.rnn para demostrar un ejemplo sencillo de cómo se pueden utilizar las RNN.

Antes de empezar a construir el modelo, vamos a utilizar una función incorporada en PyTorch para comprobar el dispositivo en el que estamos ejecutando (CPU o GPU). Esta implementación no requerirá de la GPU ya que el entrenamiento es realmente sencillo. Sin embargo, a medida que avancemos hacia grandes conjuntos de datos y modelos con millones de parámetros entrenables, el uso de la GPU será muy importante para acelerar el entrenamiento.

In [ ]:
# torch.cuda.is_available() checks and returns a Boolean True if a GPU is available, else it'll return False
is_cuda = torch.cuda.is_available()

# If we have a GPU available, we'll set our device to GPU. We'll use this device variable later in our code.
if is_cuda:
    device = torch.device("cuda")
    print("GPU is available")
else:
    device = torch.device("cpu")
    print("GPU not available, CPU used")

GPU not available, CPU used


Para empezar a construir nuestro propio modelo de red neuronal, podemos definir una clase que herede la clase base de PyTorch (nn.module) para todos los módulos de red neuronal. Después de hacerlo, podemos empezar a definir algunas variables y también las capas para nuestro modelo bajo el constructor. Para este modelo, sólo utilizaremos una capa de RNN seguida de una capa totalmente conectada. La capa totalmente conectada se encargará de convertir la salida de la RNN a nuestra forma de salida deseada.

También tendremos que definir la función forward pass bajo forward() como un método de clase. El orden en que se ejecuta la función forward es secuencial, por lo que tendremos que pasar las entradas y el estado oculto inicializado a cero a través de la capa RNN primero, antes de pasar las salidas de la RNN a la capa totalmente conectada. Ten en cuenta que estamos utilizando las capas que definimos en el constructor.

El último método que tenemos que definir es el método que hemos llamado antes para inicializar el estado oculto - init_hidden(). Esto básicamente crea un tensor de ceros con la forma de nuestros estados ocultos.

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()

        # Defining some parameters
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        #Defining the layers
        # RNN Layer
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        # Fully connected layer
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        
        batch_size = x.size(0)

        #Initializing hidden state for first input using method defined below
        hidden = self.init_hidden(batch_size)

        # Passing in the input and hidden state into the model and obtaining outputs
        out, hidden = self.rnn(x, hidden)
        
        # Reshaping the outputs such that it can be fit into the fully connected layer
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)
        
        return out, hidden
    
    def init_hidden(self, batch_size):
        # This method generates the first hidden state of zeros which we'll use in the forward pass
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
         # We'll send the tensor holding the hidden state to the device we specified earlier as well
        return hidden

Después de definir el modelo anterior, tendremos que instanciar el modelo con los parámetros pertinentes y definir también nuestros hiperparámetros. Los hiperparámetros que definimos a continuación son

- *n_epochs*: Número de épocas --> Se refiere al número de veces que nuestro modelo pasará por todo el conjunto de datos de entrenamiento
- *lr*: Tasa de Aprendizaje --> Esto afecta a la tasa en la que nuestro modelo actualiza los pesos en las celdas cada vez que se realiza la retropropagación
    - Una tasa de aprendizaje menor significa que el modelo cambia los valores de los pesos con una magnitud menor
    - Una tasa de aprendizaje mayor significa que los pesos se actualizan en mayor medida en cada paso de tiempo

Al igual que en otras redes neuronales, tenemos que definir también el optimizador y la función de pérdida. Utilizaremos CrossEntropyLoss ya que el resultado final es básicamente una tarea de clasificación.

In [ ]:
# Instantiate the model with hyperparameters
model = Model(input_size=dict_size, output_size=dict_size, hidden_dim=12, n_layers=1)
# We'll also set the model to the device that we defined earlier (default is CPU)
model = model.to(device)

# Define hyperparameters
n_epochs = 200
lr=0.01

# Define Loss, Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

Ahora ya podemos empezar el entrenamiento. Como sólo tenemos unas pocas frases, este proceso de entrenamiento es muy rápido. Sin embargo, a medida que avanzemos, los conjuntos de datos más grandes y los modelos más profundos significan que los datos de entrada son mucho más grandes y el número de parámetros dentro del modelo que tenemos que calcular es mucho mayor.

In [ ]:
# Training Run
input_seq = input_seq.to(device)
for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() # Clears existing gradients from previous epoch
    #input_seq = input_seq.to(device)
    output, hidden = model(input_seq)
    output = output.to(device)
    target_seq = target_seq.to(device)
    loss = criterion(output, target_seq.view(-1).long())
    loss.backward() # Does backpropagation and calculates gradients
    optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))

Epoch: 10/200............. Loss: 2.3255
Epoch: 20/200............. Loss: 2.0853
Epoch: 30/200............. Loss: 1.8379
Epoch: 40/200............. Loss: 1.5644
Epoch: 50/200............. Loss: 1.2931
Epoch: 60/200............. Loss: 1.0367
Epoch: 70/200............. Loss: 0.8085
Epoch: 80/200............. Loss: 0.6231
Epoch: 90/200............. Loss: 0.4855
Epoch: 100/200............. Loss: 0.3744
Epoch: 110/200............. Loss: 0.2921
Epoch: 120/200............. Loss: 0.2333
Epoch: 130/200............. Loss: 0.1911
Epoch: 140/200............. Loss: 0.1637
Epoch: 150/200............. Loss: 0.1419
Epoch: 160/200............. Loss: 0.1227
Epoch: 170/200............. Loss: 0.1075
Epoch: 180/200............. Loss: 0.0957
Epoch: 190/200............. Loss: 0.0863
Epoch: 200/200............. Loss: 0.0786


Probemos ahora nuestro modelo y veamos qué tipo de salida obtendremos. Antes de eso, vamos a definir una función de ayuda para convertir la salida de nuestro modelo en texto.

In [ ]:
def predict(model, character):
    # One-hot encoding our input to fit into the model
    character = np.array([[char2int[c] for c in character]])
    character = one_hot_encode(character, dict_size, character.shape[1], 1)
    character = torch.from_numpy(character)
    character = character.to(device)
    
    out, hidden = model(character)

    prob = nn.functional.softmax(out[-1], dim=0).data
    # Taking the class with the highest probability score from the output
    char_ind = torch.max(prob, dim=0)[1].item()

    return int2char[char_ind], hidden

In [ ]:
def sample(model, out_len, start='hey'):
    model.eval() # eval mode
    start = start.lower()
    # First off, run through the starting characters
    chars = [ch for ch in start]
    size = out_len - len(chars)
    # Now pass in the previous characters and get a new one
    for ii in range(size):
        char, h = predict(model, chars)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(model, 25, 'hola'))
print(sample(model, 25, 'espero que tengas'))
print(sample(model, 25, 'esto'))
print(sample(model, 25, 'dí'))
print(sample(model, 25, 'bueno'))
print(sample(model, 25, 'que tal'))
print(sample(model, 25, 'esp'))
print(sample(model, 25, 'bien'))
print(sample(model, 25, 'que ten'))
print(sample(model, 25, 'pases'))

hola que tal             
espero que tengas un buen
estoy bien buen día      
día                      
bueno estoy bien         
que tal                  
espero que tengas un buen
bien                     
que tengas un buen día   
pases bien buen día      


Como podemos ver, el modelo es capaz de crear frases dentro de una coherencia: "bueno, estoy bien" si lo alimentamos con las palabras "bueno", acercandose bastante a lo que pretendíamos que hiciera.
Prueba a incluir ahora más frases, más palabras y pon a prueba el modelo con diferentes parámetros. 

##Referencias: 




*   Documento inspirado en los articulos y ejemplos del blog https://blog.floydhub.com/
* Doc oficial Pytorch https://pytorch.org/docs/stable/generated/torch.nn.RNN.html
* https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html
*   [A Simple Neural Network from Scratch with PyTorch and Google Colab](https://github.com/omarsar/pytorch_intro_neural_network/blob/master/nn.ipynb)



##Fin del cuaderno